# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Question 1
limit = 5
generator = square_root_generator(limit)

vals = []

for sqrt_value in generator:
    vals.append(sqrt_value)
    print(sqrt_value)
    
print(sum(vals))

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
8.382332347441762


In [15]:
# Question 2
limit = 13
generator = square_root_generator(limit)

vals = []

for sqrt_value in generator:
    vals.append(sqrt_value)

print(vals[12]) # index 12 is 13th item

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [16]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [29]:
import dlt
import duckdb

# Define the table in duckdb
generator_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Load people 1 generator
info = generator_pipeline.run(people_1(),
                              table_name="people_records",
                              write_disposition="merge")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/davewilson/Documents/zoomcamp/dlt workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707156696.659091 is LOADED and contains no failed jobs


In [30]:
conn = duckdb.connect(f"{generator_pipeline.pipeline_name}.duckdb")

people_tab = conn.sql("SELECT SUM(age) as 'Total Age' FROM people.people_records").df()
display(people_tab)

,Total Age
0,140.0


In [31]:
# Append with people 2 generator
info = generator_pipeline.run(people_2(),
                              table_name="people_records",
                              write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/davewilson/Documents/zoomcamp/dlt workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707156706.195604 is LOADED and contains no failed jobs


In [33]:
conn = duckdb.connect(f"{generator_pipeline.pipeline_name}.duckdb")

people_tab = conn.sql("SELECT SUM(age) as 'Total Age' FROM people.people_records").df()
display(people_tab)

,Total Age
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [37]:
import dlt
import duckdb

# Define the table in duckdb
generator_pipeline2 = dlt.pipeline(destination='duckdb', dataset_name='people2')

# Load people 1 generator
info = generator_pipeline2.run(people_1(),
                              table_name="people_records2",
                              write_disposition="merge",
                              primary_key="ID")

# Merge with people 2 generator
info = generator_pipeline2.run(people_2(),
                              table_name="people_records2",
                              write_disposition="merge",
                              primary_key="ID")

conn = duckdb.connect(f"{generator_pipeline2.pipeline_name}.duckdb")

# confirm we have 8 records and id 3 is age 33
people_tab2 = conn.sql("SELECT * FROM people2.people_records2").df() #SUM(age) as 'Total Age' 

people_tab2 = conn.sql("SELECT SUM(age) as 'Total Age' FROM people2.people_records2").df()
display(people_tab2)

,Total Age
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX